### Wrapper function for distributed experiments with TensorFlow

In [1]:
def wrapper(learning_rate):
    import tensorflow as tf
    import numpy as np
    from hops import tensorboard
    from hops import hdfs
    from hops import devices
    import os
    from datetime import datetime
    import pydoop.hdfs as pyhdfs
    startTime = datetime.now()

    tensorboard_logdir = tensorboard.logdir()
    project_path = hdfs.project_path()
    num_gpus = devices.get_num_gpus()
    print("Num gpus: {0}".format(num_gpus))
    print("Cuda support: {0}".format(tf.test.is_built_with_cuda()))
    # Constants
    NUM_STEPS = 2001
    BATCH_SIZE = 1024
    TEST_SIZE = 130622
    TRAIN_SIZE = 522490
    NUM_FEATURES = 3
    NUM_CLASSES = 7
    SEQUENCE_SIZE = 200
    NUM_HIDDEN_UNITS = 64
    NUM_EPOCHS = 100
    TRAIN_FEATURES_PATH = project_path + "HAR_Dataset/cleaned_data/train/features/x_train.csv"
    TRAIN_LABELS_PATH = project_path + "HAR_Dataset/cleaned_data/train/labels/y_train.csv"
    TEST_FEATURES_PATH = project_path + "HAR_Dataset/cleaned_data/test/features/x_test.csv"
    TEST_LABELS_PATH = project_path + "HAR_Dataset/cleaned_data/test/labels/y_test.csv"
    OUTPUT_PATH = project_path + "HAR_Dataset/output/local_gpu"

    # Build the function to average the gradients
    def average_gradients(tf, tower_grads):
        average_grads = []
        for grad_and_vars in zip(*tower_grads):
            # Note that each grad_and_vars looks like the following:
            #   ((grad0_gpu0, var0_gpu0), ... , (grad0_gpuN, var0_gpuN))
            grads = []
            for g, _ in grad_and_vars:
                # Add 0 dimension to the gradients to represent the tower.
                expanded_g = tf.expand_dims(g, 0)

                # Append on a 'tower' dimension which we will average over below.
                grads.append(expanded_g)

            # Average over the 'tower' dimension.
            grad = tf.concat(grads, 0)
            grad = tf.reduce_mean(grad, 0)

            # Keep in mind that the Variables are redundant because they are shared
            # across towers. So .. we will just return the first tower's pointer to
            # the Variable.
            v = grad_and_vars[0][1]
            grad_and_var = (grad, v)
            average_grads.append(grad_and_var)
        return average_grads

    def read_csv_features(features, batch_size=100):
        """ Reads CSV files from disk into TF-HDFS queues and return batch"""
        feature_queue = tf.train.string_input_producer(features, shuffle=False, capacity=100000, name="feature_queue")
        feature_reader = tf.TextLineReader(name="feature_reader")
        _, feat_csv = feature_reader.read(feature_queue)
        feature_defaults = [[1.0] for col in range(SEQUENCE_SIZE * NUM_FEATURES)]
        feature = tf.stack(tf.decode_csv(feat_csv, feature_defaults), name="input_features")
        print("features: {0}, shape: {1}".format(feature, feature.shape))
        return tf.train.batch([feature], batch_size, num_threads=1, name="batch_csv", capacity=100000)

    def read_csv_labels(labels, batch_size=10):
        """Reads CSV files from disk into TF-HDFS queues and return batch"""
        label_queue = tf.train.string_input_producer(labels, shuffle=False, capacity=100000, name="label_queue")
        label_reader = tf.TextLineReader(name="label_reader")
        _, label_csv = label_reader.read(label_queue)
        label_defaults = [tf.constant([], dtype=tf.int64)]
        label = tf.stack(tf.decode_csv(label_csv, label_defaults), name="input_labels")
        return tf.train.batch([label], batch_size, num_threads=1, name="label_batch_csv", capacity=100000)

    ################  The computational graph ########################

    # Place all ops on CPU by default
    with tf.device('/cpu:0'):
        tower_grads = []
        reuse_vars = False

        # Read input from queues
        x_train_csv = read_csv_features([TRAIN_FEATURES_PATH], BATCH_SIZE)
        y_train_csv = read_csv_labels([TRAIN_LABELS_PATH], BATCH_SIZE)
        x_test_csv = read_csv_features([TEST_FEATURES_PATH], TEST_SIZE)
        y_test_csv = read_csv_labels([TEST_LABELS_PATH], TEST_SIZE)
        x_train_csv = tf.reshape(x_train_csv, [x_train_csv.shape[0].value, SEQUENCE_SIZE, NUM_FEATURES])
        x_test_csv = tf.reshape(x_test_csv, [x_test_csv.shape[0].value, SEQUENCE_SIZE, NUM_FEATURES])

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        
        # Loop over all GPUs and construct their own computation graph
        for i in range(num_gpus):
            with tf.device('/gpu:%d' % i):
                # Split data between GPUs
                _x = x_train_csv[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]
                _y = y_train_csv[i * BATCH_SIZE: (i + 1) * BATCH_SIZE]

                # First FCC layer
                W = {
                    'hidden': tf.Variable(tf.random_normal([NUM_FEATURES, NUM_HIDDEN_UNITS])),
                    'output': tf.Variable(tf.random_normal([NUM_HIDDEN_UNITS, NUM_CLASSES]))
                }
                biases = {
                    'hidden': tf.Variable(tf.random_normal([NUM_HIDDEN_UNITS], mean=1.0)),
                    'output': tf.Variable(tf.random_normal([NUM_CLASSES]))
                }

                # Reshape for convenience
                X = tf.transpose(_x, [1, 0, 2])
                X = tf.reshape(X, [-1, NUM_FEATURES])
                X_test = tf.transpose(x_test_csv, [1, 0, 2])
                X_test = tf.reshape(X_test, [-1, NUM_FEATURES])

                # Output from first FCC layer, split into sequences for truncated backprop
                hidden = tf.nn.relu(tf.matmul(X, W['hidden']) + biases['hidden'])
                hidden = tf.split(hidden, SEQUENCE_SIZE, 0)
                hidden_test = tf.nn.relu(tf.matmul(X_test, W['hidden']) + biases['hidden'])
                hidden_test = tf.split(hidden_test, SEQUENCE_SIZE, 0)

                # Stack 2 LSTM layers
                lstm_layers = [tf.contrib.rnn.BasicLSTMCell(NUM_HIDDEN_UNITS, forget_bias=1.0) for _ in range(2)]
                lstm_layers = tf.contrib.rnn.MultiRNNCell(lstm_layers)

                # Get output from LSTM layers
                outputs, _ = tf.contrib.rnn.static_rnn(lstm_layers, hidden, dtype=tf.float32)
                outputs_test, _ = tf.contrib.rnn.static_rnn(lstm_layers, hidden_test, dtype=tf.float32)

                # Get output for the last time step
                lstm_last_output = outputs[-1]
                lstm_last_output_test = outputs_test[-1]

                # Get Logits
                logits = tf.matmul(lstm_last_output, W['output']) + biases['output']
                logits_test = tf.matmul(lstm_last_output_test, W['output']) + biases['output']

                # Make predictions with softmax + argmax
                softmax_prediction = tf.nn.softmax(logits, name="prediction")
                prediction = tf.argmax(softmax_prediction, 1)
                softmax_prediction_test = tf.nn.softmax(logits_test, name="prediction_test")
                prediction_test = tf.argmax(softmax_prediction_test, 1)

                # L2 Regularization
                L2_LOSS = 0.0015
                l2 = L2_LOSS * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())

                # Cross entropy loss + L2 regularization
                loss = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(
                        labels=tf.one_hot(tf.reshape(_y, [-1]), NUM_CLASSES),
                        logits=logits))
                loss_test = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(
                        labels=tf.one_hot(tf.reshape(y_test_csv, [-1]), NUM_CLASSES),
                        logits=logits_test))
                loss_reg = loss + l2

                # Define optimizer
                optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
                grads = optimizer.compute_gradients(loss_reg)

                # Only first GPU compute accuracy
                if i == 0:
                    # Test trained model
                    correct_prediction = tf.equal(prediction,
                                                  tf.argmax(tf.one_hot(tf.reshape(_y, [-1]), NUM_CLASSES), 1))
                    correct_prediction_test = tf.equal(prediction_test,
                                                       tf.argmax(tf.one_hot(tf.reshape(y_test_csv, [-1]), NUM_CLASSES),
                                                                 1))
                    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
                    accuracy_test = tf.reduce_mean(tf.cast(correct_prediction_test, tf.float32), name="accuracy_test")

                reuse_vars = True
                tower_grads.append(grads)

            with tf.device('/cpu:0'):
                if i == 0:
                    tf.summary.scalar("loss_test", loss_test)  # for tensorboard
                    tf.summary.scalar("acc_test", accuracy_test)  # for tensorboard

        tower_grads = average_gradients(tf, tower_grads)
        train_step = optimizer.apply_gradients(tower_grads)

        # Utility stuff tensorboard and logging
        saver = tf.train.Saver()
        summary_op = tf.summary.merge_all()
        init_op = tf.global_variables_initializer()
        summary_writer = tf.summary.FileWriter(tensorboard_logdir, graph=tf.get_default_graph())
        display_step = 200
        history = dict(test_loss=[], test_acc=[], log=[])
        
        config = tf.ConfigProto(allow_soft_placement = True)
        # Run the graph for NUM_STEPS, compute test accuracy incrementally
        with tf.Session(config=config) as sess:
            print('Initialzing training, first step takes some time to compute the accuracy...')
            sess.run(init_op)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            step = 0
            for i in range(NUM_STEPS):
                if coord.should_stop():
                    print("coord stop")
                    break
                else:
                    batch_x, batch_y = sess.run([x_train_csv, y_train_csv])
                    sess.run(train_step, feed_dict={x_train_csv: batch_x, y_train_csv: batch_y})
                    print("step: {0}".format(i))
                    #if (i % 50 == 0):
                        #print("step: {0}".format(i))
                    if (i % display_step == 0):
                        eval_x, eval_y = sess.run([x_test_csv, y_test_csv])
                        summary, test_a, test_l = sess.run([summary_op, accuracy_test, loss_test],
                                                                 feed_dict={x_test_csv: eval_x, y_test_csv: eval_y})
                        result = "step: {0}, test acc: {1}, test loss: {2}".format(i, test_a, test_l)
                        print(result)
                        hdfs.log(result)
                        history['test_loss'].append(test_a)
                        history['test_acc'].append(test_l)
                        history['log'].append(result)
                        with tf.device('/cpu:0'):
                            summary_writer.add_summary(summary, step)
                    print("")

            endTime = datetime.now()
            timeElapsed = endTime - startTime
            accs = "\n".join(str(x) for x in history["test_acc"])
            loss = "\n".join(str(x) for x in history["test_loss"])
            logs = "\n".join(str(x) for x in history["log"])
            pyhdfs.dump(accs, OUTPUT_PATH + "/accuracy")
            pyhdfs.dump(loss, OUTPUT_PATH + "/loss")
            pyhdfs.dump(logs, OUTPUT_PATH + "/log")
            time = "start: " + str(startTime) + "\nend: " + str(endTime) + "\nduration: " + str(timeElapsed)
            pyhdfs.dump(time, OUTPUT_PATH + "/time")
            coord.request_stop()
            coord.join(threads)


Starting Spark application
The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3236,application_1513605045578_0477,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


### Launch experiment

In [ ]:
from hops import util
from hops import tflauncher

args_dict = {'learning_rate': [0.0025]}

# Generate a grid for the given hyperparameters
args_dict_grid = util.grid_params(args_dict)

# Launch training
tensorboard_hdfs_logdir = tflauncher.launch(spark, wrapper, args_dict_grid)

Finished TensorFlow job 

Make sure to check /Logs/TensorFlow/application_1513605045578_0477/runId.0 for logfile and TensorBoard logdir